In [ ]:
import os
import math
import torch
from dataclasses import dataclass, field

from code.train import train
from code.optimizers import Optimizer
# from code.problems import Problem
from code.problem import Loss
from code.datasets import Dataset
from code.models import Model

# %matplotlib widget
%load_ext autoreload
%autoreload 1

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [ ]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

In [ ]:
os.path.basename(os.getcwd())

In [104]:
@dataclass
class Config():
    n_iters: int = 2000
    n_peers: int = 20
    seed: int = 0

    model: Model = field(default_factory=lambda: Model.Linear)
    loss: Loss = field(default_factory=lambda: Loss.CrossEntropy)

    dataset: Dataset = field(default_factory=lambda: Dataset.MNIST)

#     model: Model = field(default_factory=lambda: Model.Mean)
#     loss: Loss = field(default_factory=lambda: Loss.MSE)

#     dataset: Dataset = field(default_factory=lambda: Dataset.Normal)
    n_samples: int = 200
    h_ratio: float = 0.99
    mu_normal: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 20
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [105]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = True
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

Trying port 24897
test len  200
3957
full test len  3957
full test len  20


/home/nazya/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/home/nazya/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/home/nazya/anaconda3/lib/python

actual1 200
actual3 18
calc3 18
calc1 200


/home/nazya/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/home/nazya/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/home/nazya/anaconda3/lib/python

CPU times: user 93.5 ms, sys: 190 ms, total: 283 ms
Wall time: 10min 46s


Process SpawnProcess-1281:
Traceback (most recent call last):
  File "/home/nazya/MeritFed-N/code/train.py", line 73, in setup
    dist.init_process_group(backend, rank=rank, world_size=size)
  File "/home/nazya/anaconda3/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py", line 907, in init_process_group
    default_pg = _new_process_group_helper(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nazya/anaconda3/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py", line 1009, in _new_process_group_helper
    backend_class = ProcessGroupGloo(backend_prefix_store, group_rank, group_size, timeout=timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: Socket Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nazya/anaconda3/lib/python3.11/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn

In [ ]:
config = Config()
config.optimizer = Optimizer.MeritFed
config.md_full_ = True
config.md_n_iters_ = 20
config.md_lr_ = 0.01
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

In [ ]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = False
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

In [ ]:
config = Config()
config.optimizer = Optimizer.MeritFed
config.md_full_ = False
config.md_n_iters_ = 20
config.md_lr_ = 0.01
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)